# Mandatory Assignment
**Torger Bocianowski**

## Task 1

### Sets

$S=$ suppliers

$P=$ plants

### Parameters

- Distance from supplier to plant (km)
- Cost per unit on a road from supplier to plant
- Truck cost 
- Truck capacity
- Plant cost
- Plant capacity
- Yield per unit at a plant
- Supply of biomass in tons for a supplier

### Decision Variables

$b_p\in\{0,1\}=$ binary variable, whether a plant at location $p$ should be built or not.

$x_{sp}=$ tons of biomass transported from supplier $s$ to plant $p$.

$t\in\mathbb{Z}=$ integer variable, the number of trucks used.

### Formulating the Objective Function

$C_p=$ cost of opening a plant.

$d_{sp}=$ distance driven (km) from supplier $s$ to plant $p$.

$y_p=$ yield per unit at a plant $p$.

$L, U=$ loading / unloading cost

<br></br>

**Cost components**

$\sum_p C_p\cdot b_p$ whether or not we decide to build a plant $p$ and multiplying it by the opening cost of $p$.

$\sum_{sp}u_{sp}x_{sp}d_{sp}$ the unit cost times the amount biomass transported times the distance driven.

$(L+U)t$, multiplying the number of trucks used by the constant load / unload cost.

<br></br>

**Objective function**

Minimize Cost: $\space\sum_p C_p\cdot b_p+\sum_{sp}u_{sp}x_{sp}d_{sp}+(L+U)t$

<p style="color: red">constraints</p>

### Constraints

- $\sum_{sp}x_{sp}y_p\geq500\space000\space000$

- $\sum_s x_{sp}y_p\leq cap_p\quad\forall p\in P$

- $\frac{x_{sp}}{cap_t}\leq t\quad\forall s\in S\quad\forall p\in P$

- $x_{sp} >= 0\quad\forall s\in S\quad\forall p\in P$

In [1]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

In [2]:
# Dataframes
suppliers_df = pd.read_csv("data/suppliers.csv")
plants_df = pd.read_csv("data/plants.csv")
roads_df = pd.read_csv("data/roads_s_p.csv")

# Sets
suppliers = suppliers_df.set_index("supplier")
suppliers = suppliers.to_dict(orient="index")

plants = plants_df.set_index("plant")
plants = plants.to_dict(orient="index")

### Defining the Model

In [3]:
gp.setParam("OutputFlag", 0)
m = gp.Model("SupplyChain")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-24


### Parameters

In [4]:
def param_from(df: pd.DataFrame, param: str) -> dict:
    df = df[["supplier", "plant", param]]
    df = df.set_index(["supplier", "plant"])
    df = df.to_dict(orient="index")
    return {(s, p): df[s, p][param] for s in suppliers for p in plants}

dist_s_p = param_from(roads_df, "dist_s_p")
unit_cost_s_p = param_from(roads_df, "cost_per_unit_s_p")
truck_cost_s_p = param_from(roads_df, "truck_cost_s_p")
truck_cap_s_p = param_from(roads_df, "truck_cap_s_p")

### Decision Variables

In [5]:
build = m.addVars(plants, vtype=GRB.BINARY, name="build")
trucks = m.addVar(vtype=GRB.INTEGER, name="trucks")
biomass = m.addVars(suppliers, plants, name="biomass")

### Objective Function

In [6]:
load, unload = 5000, 5000

m.setObjective(
    gp.quicksum(
        plants[p]["plant_cost"] * build[p]
        for p in plants
    )
    + gp.quicksum(
        unit_cost_s_p[s, p] * biomass[s, p] * dist_s_p[s, p]
        for s in suppliers
        for p in plants
    )
    + (load + unload) * trucks, 
    GRB.MINIMIZE,
)

### Constraints

In [7]:
PRODUCTION = 500_000_000

m.addConstrs(
    (
        gp.quicksum(biomass[s, p] for p in plants) <= suppliers[s]["supply"]
        for s in suppliers
    ),
    name="supply",
)

m.addConstrs(
    (
        gp.quicksum(biomass[s, p] for s in suppliers) <= plants[p]["plant_cap"] * build[p]
        for p in plants
    ),
    name="capacity",
)

m.addConstr(
    (
        gp.quicksum(biomass[s, p] * plants[p]["yield_per_unit"] 
                    for s in suppliers 
                    for p in plants
                    ) >= PRODUCTION
    ),
    name="production",
)

m.addConstrs(
    (
        biomass[s, p] / truck_cap_s_p[s, p] <= trucks
        for s in suppliers
        for p in plants
    ),
    name="trucks",
)

m.addConstrs(
    (
        biomass[s, p] >= 0
        for s in suppliers
        for p in plants
    ),
    name="non_negative_biomass",
)

m.setParam("OutputFlag", 0)

### Solve the Model

In [8]:
m.setParam("OutputFlag", 1)
m.optimize()

Set parameter OutputFlag to value 1
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 85258 rows, 42586 columns and 254675 nonzeros
Model fingerprint: 0x07a443fd
Variable types: 42418 continuous, 168 integer (167 binary)
Coefficient statistics:
  Matrix range     [2e-03, 2e+08]
  Objective range  [1e+02, 1e+08]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e-01, 5e+08]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.


Presolve removed 43426 rows and 1002 columns
Presolve time: 0.12s
Presolved: 41832 rows, 41584 columns, 207247 nonzeros
Variable types: 41416 continuous, 168 integer (167 binary)
Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.00s

Solved with dual simplex

Root relaxation: objective 6.541699e+09, 124 iterations, 0.12 seconds (0.16 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 6.5417e+09    0   21          - 6.5417e+09      -     -    0s
H    0     0                    9.068403e+09 6.5417e+09  27.9%     -    0s
H    0     0                    8.686555e+09 7.6312e+09  12.1%     -    0s
     0     0 7.6312e+09    0   23 8.6866e+09 7.6312e+09  12.1%     -    0s
     0     0 7.6323e+09    0   23 8.6866e+09 7.6323e+09  12.1%     -    0s
     0     0 7.9644e+09    0   31 8.6866e+09 7.9644e+09  8.31

In [9]:
locations = [p for p in plants if build[p].x == 1]
total_biomass = sum(biomass[s, p].x for s in suppliers for p in plants)
formatted_total_biomass = f"{total_biomass:,.0f}"
formatted_total_cost = f"${m.objVal:,.2f}"

print("RESULTS")
print(f"({len(locations)}) Locations: {locations}")
print(f"Total biomass: {formatted_total_biomass} Mg")
print(f"Trucks needed: {trucks.x}")
print(f"Total cost: {formatted_total_cost}")


RESULTS
(11) Locations: [541, 9063, 9071, 9102, 9107, 9155, 9178, 9183, 9203, 10058, 10066]
Total biomass: 2,155,172 Mg
Trucks needed: 91.0
Total cost: $8,458,593,644.09


### Interpreting the Results

As shown above, 
- there are **11 locations** at which we want to **build plants**, 
- we need **91 trucks** and 
- the **total cost** is almost **8.5 billion** dollars

## Task 2

### Sets
$S=$ suppliers

$P=$ plants

$H=$ hubs

### Parameters

- Hub cost
- Hub capacity
- Truck cost from supplier to hub
- Truck capacity from supplier to hub
- Train cost from hub to plant
- Train capacity from hub to plant
- Distance from supplier to hub
- Distance from hub to supplier
- Cost per unit on a road from supplier to hub
- Cost per unit on a road from hub to plant


### Decision Variables

$b_p\in\{0,1\}=$ binary variable, whether a plant at location $p$ should be built or not.

$b_h\in\{0,1\}=$ binary variable, whether a hub at location $h$ should be built or not.

$x_{sh}=$ tons of biomass transported from supplier $s$ to hub $h$.

$x_{hp}=$ tons of biomass transported from hub $h$ to plant $p$.

$t\in\mathbb{Z}=$ integer variable, the number of trucks to be used.

$r\in\mathbb{Z}=$ integer variable, the number of trains to be used.

### Formulating the Objective Function

$O_p=$ constant opening cost of a plant $p$.

$O_h=$ constant opening cost of a hub $h$.

$d_{sh}=$ distance driven (km) from supplier $s$ to hub $h$ by road.

$d_{hp}=$ distance driven (km) from hub $h$ to plant $p$ by train.

$L_t, U_t=$ truck loading / unloading cost

$L_r, U_r=$ train loading / unloading cost

<br></br>

**Objective function**

_Minimize Cost_: $\sum_p C_p\cdot b_p$

$\hspace{5em}+\sum_p C_h\cdot b_h$

$\hspace{5em}+\sum_{sh}u_{sh}x_{sh}d_{sh}$

$\hspace{5em}+\sum_{hp}u_{hp}x_{hp}d_{hp}$

$\hspace{5em}+(L_t+U_t)t_{sh}$

$\hspace{5em}+(L_r+U_r)r_{hp}$

<br></br>

$\hspace{2.85em}$ _s.t._: $\space\sum_{h}x_{sh}\cdot b_h\leq S_s
\hspace{2em}\forall s\in S\quad$ (supply)

$\hspace{5em} \sum_{hp}x_{hp}\cdot y_p\leq500\space000\space000$ (production)

$\hspace{5em} x_{sh}\leq t_{sh}\cdot C_{psh}$ (number of trucks)

$\hspace{5em} x_{hp}\leq r_{hp}\cdot C_{rhp}$ (number of trains)

$\hspace{5em} \sum_{sh}x_{sh}\leq$


### Constraints

<p style="color: red;">Flow balance?</p>

In [10]:
# Dataframes
hubs_df = pd.read_csv("data/hubs.csv")
railroads_df = pd.read_csv("data/railroads_h_p.csv")
roads_df = pd.read_csv("data/roads_s_h.csv")

# Sets
hubs = hubs_df.set_index("hub")
hubs = hubs.to_dict(orient="index")

### Defining the Model

In [11]:
m = gp.Model("SupplyChain2")
gp.setParam("OutputFlag", 0)

### Parameters

In [12]:
# Helper functions
def sh_param(df: pd.DataFrame, param: str) -> dict:
    df = df[["supplier", "hub", param]]
    df = df.set_index(["supplier", "hub"])
    df = df.to_dict(orient="index")
    return {(s, h): df[s, h][param] for s in suppliers for h in hubs}

def hp_param(df: pd.DataFrame, param: str) -> dict:
    df = df[["hub", "plant", param]]
    df = df.set_index(["hub", "plant"])
    df = df.to_dict(orient="index")
    return {(h, p): df[h, p][param] for h in hubs for p in plants}

# params from roads_s_h.csv
dist_s_h = sh_param(roads_df, "dist_s_h")
unit_cost_s_h = sh_param(roads_df, "cost_per_unit_s_h")
truck_cap_s_h = sh_param(roads_df, "truck_cap_s_h")

# params from railroads_h_p.csv
dist_h_p = hp_param(railroads_df, "dist_h_p")
unit_cost_h_p = hp_param(railroads_df, "cost_per_unit_h_p")
train_cap_h_p = hp_param(railroads_df, "train_cap_h_p")

### Decision Variables

In [13]:
build_plant = m.addVars(plants, vtype=GRB.BINARY, name="build_plant")
build_hub = m.addVars(hubs, vtype=GRB.BINARY, name="build_hub")

trucks = m.addVars(suppliers, hubs, vtype=GRB.INTEGER, name="trucks")
trains = m.addVars(hubs, plants, vtype=GRB.INTEGER, name="trains")

biomass_s_h = m.addVars(suppliers, hubs, name="biomass_s_h")
biomass_h_p = m.addVars(hubs, plants, name="biomass_h_p")

### Objective Function

In [14]:
truck_L, truck_U = 5000, 5000
train_L, train_U = 30_000, 30_000

m.setObjective(
    gp.quicksum(
        plants[p]["plant_cost"] * build_plant[p]
        for p in plants
    )
    + gp.quicksum(
        hubs[h]["hub_cost"] * build_hub[h]
        for h in hubs
    )
    + gp.quicksum(
        unit_cost_s_h[s, h] * biomass_s_h[s, h] * dist_s_h[s, h] 
                        + ((truck_L + truck_U) * trucks[s, h])
        for s in suppliers
        for h in hubs
    )
    + gp.quicksum(
        unit_cost_h_p[h, p] * biomass_h_p[h, p] * dist_h_p[h, p] 
                          + ((train_L + train_U) * trains[h, p])
        for h in hubs
        for p in plants
    ),
    GRB.MINIMIZE,
)

### Constraints

In [15]:
PRODUCTION = 500_000_000

m.addConstrs(
    (
        gp.quicksum(biomass_s_h[s, h] * build_hub[h] for h in hubs) <= suppliers[s]["supply"]
        for s in suppliers
    ),
    name="supply",
)

m.addConstr(
    (
        gp.quicksum(biomass_h_p[h, p] * plants[p]["yield_per_unit"]
                    for h in hubs 
                    for p in plants
                    ) >= PRODUCTION
    ),
    name="production",
)

m.addConstrs(
    (
        biomass_s_h[s, h] <= trucks[s, h] * truck_cap_s_h[s, h]
        for s in suppliers
        for h in hubs
    ),
    name="trucks",
)

m.addConstrs(
    (
        biomass_h_p[h, p] <= trains[h, p] * train_cap_h_p[h, p]
        for h in hubs
        for p in plants
    ),
    name="trains",
)

m.addConstrs(
    (
        gp.quicksum(biomass_s_h[s, h] for s in suppliers) <= hubs[h]["hub_cap"] * build_hub[h]
        for h in hubs
    ),
    name="hub_capacity",
)


m.addConstrs(
    (
        gp.quicksum(biomass_h_p[h, p] for h in hubs) <= plants[p]["plant_cap"] * build_plant[p]
        for p in plants
    ),
    name="plant_capacity",
)

m.addConstrs(
    (
        gp.quicksum(biomass_s_h[s, h] for s in suppliers)
        == gp.quicksum(biomass_h_p[h, p] for p in plants)
        for h in hubs
    ),
    name="balance_flow",
)

gp.setParam("OutputFlag", 0)

### Solve the Model

In [16]:
m.optimize()

### Solve the Model

In [17]:
plants_locations = [p for p in plants if build_plant[p].x == 1]
hubs_locations = [h for h in hubs if build_hub[h].x == 1]

total_biomass_s_h = sum(biomass_s_h[s, h].x for s in suppliers for h in hubs)
total_biomass_s_h = f"{total_biomass_s_h:,.0f}"

total_biomass_h_p = sum(biomass_h_p[h, p].x for h in hubs for p in plants)
total_biomass_h_p = f"{total_biomass_h_p:,.0f}"

if total_biomass_s_h != total_biomass_h_p:
    print(f"Biomass from s to h: {total_biomass_s_h} Mg")
    print(f"Biomass from h to p: {total_biomass_h_p} Mg")
    raise Exception("Biomass from s to h must be equal to biomass from h to p")


print("RESULTS\n")
print(f" ({len(plants_locations)}) Plants: {plants_locations}")
print(f"({len(hubs_locations)})   Hubs: {hubs_locations[:10]}")
print(f"\t     {hubs_locations[10:20]}")
print(f"\t     {hubs_locations[20:]}\n")
print(f"Total biomass: {total_biomass_s_h} Mg")

print(f"Trucks needed: {len(trucks.keys())}")
print(f"Trains needed: {len(trains.keys())}")
print(f"\nTotal cost: ${m.objVal:,.2f}")

RESULTS

 (8) Plants: [541, 9047, 9060, 9091, 9178, 9183, 9203, 10066]
(31)   Hubs: [17201, 17218, 17359, 17372, 17395, 17404, 17447, 17466, 17507, 17592]
	     [17620, 17679, 17717, 17784, 17792, 17822, 17829, 17896, 17931, 17934]
	     [17942, 17943, 18029, 18042, 18063, 18082, 18127, 18286, 18288, 18294, 18303]

Total biomass: 2,155,172 Mg
Trucks needed: 8382
Trains needed: 5511

Total cost: $5,135,420,807.27


### Interpreting the Results

As shown above, 
- there are **8 locations** at which we want to **build plants**, 
- there are **31 locations** at which we want to **build hubs**, 
- we need **8382 trucks** and **5511** trains,
- the **total cost** is about **5.1 billion** dollars

## Task 3

### Sets

$S=$ suppliers

$P=$ plants

### Parameters

- Distance from supplier to plant (km)
- Cost per unit on a road from supplier to plant
- Truck cost 
- Truck capacity
- Plant cost
- Plant capacity
- Yield per unit at a plant
- Supply of biomass in tons for a supplier

### Decision Variables

$b_p\in\{0,1\}=$ binary variable, whether a plant at location $p$ should be built or not.

$x_{sp}=$ tons of biomass transported from supplier $s$ to plant $p$.

$t\in\mathbb{Z}=$ integer variable, the number of trucks used.

### Defining the Model

In [25]:
m = gp.Model("SupplyChain3")

### Parameters

In [26]:
# Same as before, but with a third party with unlimited supply
unit_cost_third_party = 2000

### Decision Variables

In [27]:
build_plant = m.addVars(plants, vtype=GRB.BINARY, name="build_plant")
build_hub = m.addVars(hubs, vtype=GRB.BINARY, name="build_hub")

trucks = m.addVars(suppliers, hubs, vtype=GRB.INTEGER, name="trucks")
trains = m.addVars(hubs, plants, vtype=GRB.INTEGER, name="trains")
third_party_suppliers = m.addVar(vtype=GRB.INTEGER, name="third_party_suppliers")

biomass_s_h = m.addVars(suppliers, hubs, name="biomass_s_h")
biomass_h_p = m.addVars(hubs, plants, name="biomass_h_p")

### Objective Function

In [28]:
truck_L, truck_U = 5000, 5000
train_L, train_U = 30_000, 30_000
third_party_unit_cost = 2000

# same as before but with third party with unlimited supply to hubs and a unit cost of 2000
# the location of the third parties does not matter
m.setObjective(
    gp.quicksum(
        plants[p]["plant_cost"] * build_plant[p]
        for p in plants
    )
    + gp.quicksum(
        hubs[h]["hub_cost"] * build_hub[h]
        for h in hubs
    )
    + gp.quicksum(
        unit_cost_s_h[s, h] * biomass_s_h[s, h] * dist_s_h[s, h] 
                        + ((truck_L + truck_U) * trucks[s, h])
        for s in suppliers
        for h in hubs
    )
    + gp.quicksum(
        unit_cost_h_p[h, p] * biomass_h_p[h, p] * dist_h_p[h, p] 
                          + ((train_L + train_U) * trains[h, p])
        for h in hubs
        for p in plants
    )
    + third_party_unit_cost * third_party_suppliers,
    GRB.MINIMIZE,
)

### Constraints

In [92]:
# new set: third party supply locations
third_party = {
    69: {
        "supply": float("inf"),
    }
}

m = gp.Model("SupplyChain3")

unit_cost_third_party = 2000

build_plant = m.addVars(plants, vtype=GRB.BINARY, name="build_plant")
build_hub = m.addVars(hubs, vtype=GRB.BINARY, name="build_hub")

trucks = m.addVars(suppliers, hubs, vtype=GRB.INTEGER, name="trucks")
trains = m.addVars(hubs, plants, vtype=GRB.INTEGER, name="trains")

biomass_s_h = m.addVars(suppliers, hubs, name="biomass_s_h")
biomass_h_p = m.addVars(hubs, plants, name="biomass_h_p")
third_party_biomass = m.addVars(third_party, hubs, name="third_party_biomass")

truck_L, truck_U = 5000, 5000
train_L, train_U = 30_000, 30_000
third_party_unit_cost = 2000

m.setObjective(
    gp.quicksum(
        plants[p]["plant_cost"] * build_plant[p]
        for p in plants
    )
    + gp.quicksum(
        hubs[h]["hub_cost"] * build_hub[h]
        for h in hubs
    )
    + gp.quicksum(
        unit_cost_s_h[s, h] * biomass_s_h[s, h] * dist_s_h[s, h] 
                        + ((truck_L + truck_U) * trucks[s, h])
        for s in suppliers
        for h in hubs
    )
    + gp.quicksum(
        unit_cost_h_p[h, p] * biomass_h_p[h, p] * dist_h_p[h, p] 
                          + ((train_L + train_U) * trains[h, p])
        for h in hubs
        for p in plants
    )
    + gp.quicksum(
        unit_cost_third_party * third_party_biomass[t, h]
        for t in third_party
        for h in hubs
    ),
    GRB.MINIMIZE,
)

PRODUCTION = 800_000_000

m.addConstrs(
    (
        gp.quicksum(
            biomass_s_h[s, h] * build_hub[h] for h in hubs
            ) <= suppliers[s]["supply"]
        for s in suppliers
    ),
    name="supply",
)

m.addConstr(
    gp.quicksum(
        biomass_h_p[h, p] * plants[p]["yield_per_unit"]
                for h in hubs 
                for p in plants
        ) >= PRODUCTION,
    name="production",
)

m.addConstrs(
    (
        biomass_s_h[s, h] <= trucks[s, h] * truck_cap_s_h[s, h]
        for s in suppliers
        for h in hubs
    ),
    name="trucks",
)

m.addConstrs(
    (
        biomass_h_p[h, p] <= trains[h, p] * train_cap_h_p[h, p]
        for h in hubs
        for p in plants
    ),
    name="trains",
)

m.addConstrs(
    (
        gp.quicksum(biomass_s_h[s, h] for s in suppliers) 
        + third_party_biomass[t, h]
        <= hubs[h]["hub_cap"] * build_hub[h]
        for h in hubs
        for t in third_party
    ),
    name="hub_capacity",
)

m.addConstrs(
    (
        gp.quicksum(biomass_h_p[h, p] for h in hubs) <= plants[p]["plant_cap"] * build_plant[p]
        for p in plants
    ),
    name="plant_capacity",
)

m.addConstrs(
    (
        gp.quicksum(biomass_s_h[s, h] for s in suppliers) 
        + gp.quicksum(third_party_biomass[t, h] for t in third_party)
        == gp.quicksum(biomass_h_p[h, p] for p in plants)
        for h in hubs
        for t in third_party
    ),
    name="balance_flow",
)

m.setParam("OutputFlag", 0)

{(17201, 69): <gurobi.Constr *Awaiting Model Update*>,
 (17218, 69): <gurobi.Constr *Awaiting Model Update*>,
 (17359, 69): <gurobi.Constr *Awaiting Model Update*>,
 (17372, 69): <gurobi.Constr *Awaiting Model Update*>,
 (17395, 69): <gurobi.Constr *Awaiting Model Update*>,
 (17404, 69): <gurobi.Constr *Awaiting Model Update*>,
 (17447, 69): <gurobi.Constr *Awaiting Model Update*>,
 (17466, 69): <gurobi.Constr *Awaiting Model Update*>,
 (17507, 69): <gurobi.Constr *Awaiting Model Update*>,
 (17592, 69): <gurobi.Constr *Awaiting Model Update*>,
 (17620, 69): <gurobi.Constr *Awaiting Model Update*>,
 (17679, 69): <gurobi.Constr *Awaiting Model Update*>,
 (17717, 69): <gurobi.Constr *Awaiting Model Update*>,
 (17784, 69): <gurobi.Constr *Awaiting Model Update*>,
 (17792, 69): <gurobi.Constr *Awaiting Model Update*>,
 (17822, 69): <gurobi.Constr *Awaiting Model Update*>,
 (17829, 69): <gurobi.Constr *Awaiting Model Update*>,
 (17896, 69): <gurobi.Constr *Awaiting Model Update*>,
 (17931, 6

In [93]:
m.optimize()
gp.setParam("OutputFlag", 1)


Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 14127 rows, 28019 columns and 61349 nonzeros
Model fingerprint: 0x903a1060
Model has 254 quadratic constraints
Variable types: 13926 continuous, 14093 integer (200 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+08]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [8e+00, 1e+08]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e+08, 8e+08]
  QRHS range       [8e-01, 5e+04]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve time: 0.05s
Presolved: 22763 rows, 36401 columns, 94877 nonzeros
Variable types: 22308 continuous, 14093 integer (200 binary)
Found heuristic solution: objective 7.978673e+09

Root relaxation: objective 6.815958e+09, 489 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    | 

In [95]:
plants_locations = [p for p in plants if build_plant[p].x == 1]
hubs_locations = [h for h in hubs if build_hub[h].x == 1]

total_biomass_s_h = sum(biomass_s_h[s, h].x for s in suppliers for h in hubs)
total_biomass_s_h = f"{total_biomass_s_h:,.0f}"

total_biomass_h_p = sum(biomass_h_p[h, p].x for h in hubs for p in plants)
total_biomass_h_p = f"{total_biomass_h_p:,.0f}"

total_biomass_third_party = sum(third_party_biomass[t, h].x for t in third_party for h in hubs)
total_biomass_third_party = f"{total_biomass_third_party:,.0f}"

print("RESULTS\n")
print(f" ({len(plants_locations)}) Plants: {plants_locations}")
print(f"({len(hubs_locations)})   Hubs: {hubs_locations[:9]}")
print(f"\t     {hubs_locations[9:]}")
print(f"Total biomass sh: {total_biomass_s_h} Mg")
print(f"Total biomass hp: {total_biomass_h_p} Mg")
print(f"Total biomass third: {total_biomass_third_party} Mg")

print(f"Trucks needed: {len(trucks.keys())}")
print(f"Trains needed: {len(trains.keys())}")
print(f"Total cost: ${m.objVal:,.2f}")


RESULTS

 (6) Plants: [9044, 9047, 9178, 9183, 9203, 10066]
(18)   Hubs: [17201, 17218, 17359, 17447, 17507, 17592, 17679, 17792, 17829]
	     [17896, 17934, 17943, 18029, 18042, 18082, 18286, 18288, 18294]
Total biomass sh: 1,401,295 Mg
Total biomass hp: 3,448,276 Mg
Total biomass third: 2,046,981 Mg
Trucks needed: 8382
Trains needed: 5511
Total cost: $7,139,972,964.32
